# Definição do problema: Prever se um cliente está satisfeito

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.7


In [2]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [26]:
# Imports
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from imblearn.over_sampling import SMOTE

In [4]:
# Criando o Spark Context
sc = SparkContext(appName = "prjt3")

In [5]:
sc.setLogLevel("ERROR")

# Criando a sessão
spark = SparkSession.builder.getOrCreate()
spark

## Carregando o Dataset

In [6]:
# Carrega os dados
dados = spark.read.csv('train.csv', inferSchema = True, header = True)

In [7]:
# Número de registros
dados.count()

76020

In [8]:
# Visualiza os dados no padrão do Spark DataFrame
dados.show(10)

+---+----+-----+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------+------------------+----------+--------+----------+--------+----------+--------+----------+--------+----------+--------+-----------+---------+-----------+-----------------+---------------+-----------------+---------------+-----------------+---------------+---------+-----------+---------+-----------+---------+-----------+---------+---------+-----------+---------+-----------+---------+-------------+-----------+-------------+---------+-----------+---------+-----------+-----------+---------+---------+-----------+---------+-----------+---------+-----------+---------+-------------+-----------+---------+-----------+--------

In [9]:
# Visualiza os dados no formato do Pandas
dados.limit(10).toPandas()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
5,13,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87975.750000,0
6,14,2,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94956.660000,0
7,18,2,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,251638.950000,0
8,20,2,45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101962.020000,0
9,23,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356463.060000,0


In [10]:
# Schema
dados.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- var3: integer (nullable = true)
 |-- var15: integer (nullable = true)
 |-- imp_ent_var16_ult1: double (nullable = true)
 |-- imp_op_var39_comer_ult1: double (nullable = true)
 |-- imp_op_var39_comer_ult3: double (nullable = true)
 |-- imp_op_var40_comer_ult1: double (nullable = true)
 |-- imp_op_var40_comer_ult3: double (nullable = true)
 |-- imp_op_var40_efect_ult1: double (nullable = true)
 |-- imp_op_var40_efect_ult3: double (nullable = true)
 |-- imp_op_var40_ult1: double (nullable = true)
 |-- imp_op_var41_comer_ult1: double (nullable = true)
 |-- imp_op_var41_comer_ult3: double (nullable = true)
 |-- imp_op_var41_efect_ult1: double (nullable = true)
 |-- imp_op_var41_efect_ult3: double (nullable = true)
 |-- imp_op_var41_ult1: double (nullable = true)
 |-- imp_op_var39_efect_ult1: double (nullable = true)
 |-- imp_op_var39_efect_ult3: double (nullable = true)
 |-- imp_op_var39_ult1: double (nullable = true)
 |-- imp_sal_var16_ult1: dou

In [11]:
# Separamos os dados ausentes (se existirem) e removemos (se existirem)
dados_com_linhas_removidas = dados.na.drop()
print('Número de linhas antes de remover valores ausentes:', dados.count())
print('Número de linhas após remover valores ausentes:', dados_com_linhas_removidas.count())

Número de linhas antes de remover valores ausentes: 76020
Número de linhas após remover valores ausentes: 76020


In [12]:
dados = dados.drop(dados.ID)

In [13]:
dados.groupBy("var3").count().orderBy("var3").show()

+-------+-----+
|   var3|count|
+-------+-----+
|-999999|  116|
|      0|   75|
|      1|  105|
|      2|74165|
|      3|  108|
|      4|   86|
|      5|   63|
|      6|   82|
|      7|   97|
|      8|  138|
|      9|  110|
|     10|   72|
|     11|   66|
|     12|   85|
|     13|   98|
|     14|   61|
|     15|   34|
|     16|    9|
|     17|    7|
|     18|   10|
+-------+-----+
only showing top 20 rows



In [14]:
dados.groupBy("var36").count().show()

+-----+-----+
|var36|count|
+-----+-----+
|    1|14664|
|    3|22177|
|    2| 8704|
|   99|30064|
|    0|  411|
+-----+-----+



In [15]:
dados.groupBy("var21").count().show()

+-----+-----+
|var21|count|
+-----+-----+
| 3000|   84|
| 1500|   31|
| 1800|  206|
| 4500|   96|
| 3300|    2|
|  900|  236|
| 1200|   12|
| 7200|   62|
| 3600|   52|
| 6000|   27|
| 9000|   14|
| 2400|    3|
|10500|    1|
|    0|75152|
| 2700|   26|
| 5100|    2|
| 7500|    1|
| 6600|    1|
| 2100|    2|
| 5400|    4|
+-----+-----+
only showing top 20 rows



In [16]:
dados.groupBy("var38").count().show()

+---------+-----+
|    var38|count|
+---------+-----+
|108845.91|    1|
| 38272.44|    1|
|138101.94|    1|
| 23894.97|    1|
|258941.73|    1|
| 94576.56|    1|
| 60170.88|    3|
|  38587.5|    1|
| 42821.79|    1|
|153463.83|    1|
|111146.04|    1|
|266319.51|    2|
| 72774.12|    1|
| 43179.75|    1|
|134509.71|    1|
|133824.78|    2|
| 76321.14|    1|
|103554.93|    1|
|198463.14|    1|
|256363.47|    1|
+---------+-----+
only showing top 20 rows



In [17]:
# Lista de variáveis de entrada (todas menos a última)
variaveis_entrada = dados.columns[:-1]

In [18]:
assembler = VectorAssembler().setInputCols(variaveis_entrada).setOutputCol("features")
output = assembler.transform(dados).select("features", 'TARGET')
scaler = StandardScaler().setInputCol("features").setOutputCol("scaledFeatures").setWithStd(True).setWithMean(False)
scalerModel = scaler.fit(output)
scaledData = scalerModel.transform(output).select("scaledFeatures", 'TARGET')
pca = PCA().setInputCol("scaledFeatures").setOutputCol("pcaFeatures").setK(5).fit(scaledData)
pcaDados = pca.transform(scaledData).select("pcaFeatures", 'TARGET')
pcaDados.show()
pcaDados.head(1)

+--------------------+------+
|         pcaFeatures|TARGET|
+--------------------+------+
|[0.51318271399369...|     0|
|[-1.2006486738358...|     0|
|[-0.1756015041533...|     0|
|[-6.6039314918786...|     0|
|[-4.2421084796185...|     0|
|[0.51297389903558...|     0|
|[-0.3692703297821...|     0|
|[-0.3065398322083...|     0|
|[0.42934077498377...|     0|
|[0.20745946968436...|     0|
|[-11.395458744975...|     0|
|[0.50163152683831...|     0|
|[-0.4421819178737...|     0|
|[-4.9585149488386...|     0|
|[-34.366102122896...|     0|
|[-0.2485399413801...|     0|
|[0.98990963904744...|     0|
|[-4.7700059810275...|     0|
|[-2.4970563333163...|     0|
|[-0.3271070611871...|     0|
+--------------------+------+
only showing top 20 rows



[Row(pcaFeatures=DenseVector([0.5132, -1.7794, -0.4578, -1.0572, 0.704]), TARGET=0)]

In [19]:
# Lista a correlação entre os atributos e a variável alvo
for item in (Correlation.corr(pcaDados, 'pcaFeatures', 'pearson').collect()[0][0].toArray()):
    print(item[4])

-9.662102022464044e-15
4.423785609631877e-15
-1.996593505340427e-15
6.030816937706429e-15
1.0


In [20]:
dados_pandas = pcaDados.toPandas()
dados_pandas.TARGET.value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [35]:
# Como não identifiquei uma forma de balancear a variável alvo, irei refazer o processo em pandas.
dados_pandas = dados.toPandas()
dados_pandas

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,0
76016,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,0
76017,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,0
76018,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,0


In [64]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
import pandas as pd

# Cria um objeto separado para a variável target
y_treino = dados_pandas['TARGET']
# Cria um objeto separadado para as variáveis de entrada
X_treino = dados_pandas.drop('TARGET', axis = 1)

#Aplica a técnica de oversampling e aumentar o número de exemplos da classe minoritária
over_sampler = SMOTE(k_neighbors = 2)
X_treino, y_treino = over_sampler.fit_resample(X_treino, y_treino)

#Aplicar a técnica de padronização dos dados
scaler = StandardScaler()
scaler.fit(X_treino)
X_treino = scaler.transform(X_treino)
#X_teste = scaler.transform(X_teste)

#Aplicar a técnica de redu;áo de dimensionalidade nos dados
pca = PCA(n_components=5)
X_treino = pca.fit_transform(X_treino)

#Remonta o dataframe
principalDf = pd.DataFrame(data = X_treino)
X_treino_pd = pd.concat([principalDf, y_treino], axis = 1)

#Transforma em datafram do spark
X_treino = spark.createDataFrame(X_treino_pd)
X_treino.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+------+
|                  0|                   1|                   2|                   3|                   4|TARGET|
+-------------------+--------------------+--------------------+--------------------+--------------------+------+
| -2.456171337846136| -1.2259361566534486|  -0.109688137901692|-0.05134435136719607| 0.34562680066427204|     0|
|-0.6626937515989535|  10.759481337754007| -5.5143330546367135|  -4.901243187236635|   5.013044746297762|     0|
|-1.5771283959243234| -0.3507992911812729| 0.08000910791333184| 0.18122728051346434|-0.04289240144624558|     0|
|   5.27950968952817| -0.5905861507674804| 0.17394812828015843| -0.4322359826385947| -1.2038314254947724|     0|
|  2.526587042188236|  10.909687672175398|   4.909173376856295|   9.755768056029192| -11.900355780732582|     0|
|-2.4556474373673014| -1.2219832670480755|-0.10891372879885909|-0.05119132778804...| 0.346327568

In [68]:
# Lista de variáveis de entrada (todas menos a última)
assembler = VectorAssembler().setInputCols(X_treino.columns[:-1]).setOutputCol("features")
X_treino = assembler.transform(X_treino).select("features", 'TARGET')
X_treino.show()

IllegalArgumentException: Output column features already exists.

In [74]:
# Carrega os dados de teste
dados_teste = spark.read.csv('test.csv', inferSchema = True, header = True)
dados_pandas_teste = dados_teste.toPandas()

# Cria um objeto separado para a variável target
y_teste = dados_pandas_teste['TARGET']
# Cria um objeto separadado para as variáveis de entrada
X_teste = dados_pandas_teste.drop('TARGET', axis = 1)

#Aplicar a técnica de padronização dos dados
X_teste = scaler.transform(X_teste)

#Aplicar a técnica de redu;áo de dimensionalidade nos dados
X_teste = pca.transform(X_teste)

#Remonta o dataframe
Df = pd.DataFrame(data = X_teste)
teste_pd = pd.concat([Df, y_teste], axis = 1)

#Transforma em datafram do spark
X_teste = spark.createDataFrame(teste_pd)
X_teste.show()

KeyError: 'TARGET'